In [1]:
import xarray as xr
import pandas as pd
import numpy as np

era5 = xr.open_dataset('era5/era5-14param-2022-2023.nc')
terra_2022 = xr.open_dataset('modis-datasets/g4.areaAvgTimeSeries.MOD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean.20220101-20221231.51E_35N_52E_36N.nc')
terra_2023 = xr.open_dataset('modis-datasets/g4.areaAvgTimeSeries.MOD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean.20230101-20231231.51E_35N_52E_36N.nc')
aqua_2022 = xr.open_dataset('modis-datasets/g4.areaAvgTimeSeries.MYD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean.20220101-20221231.51E_35N_52E_36N.nc')
aqua_2023 = xr.open_dataset('modis-datasets/g4.areaAvgTimeSeries.MYD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean.20230101-20231231.51E_35N_52E_36N.nc')


In [2]:

#Calculate era5 average for the 1 sq degree area centered at lat,lon: 35.5,51.5
era5_avg = xr.Dataset(
    coords ={
        'latitude' : 35.5,
        'longitude' : 51.5
    }
)
# average over axis 1,2 : lat, lon
for var_name in era5.data_vars:
    var = era5[var_name]
    var_avg = var.mean(axis=(1,2))
    var_avg.attrs = var.attrs
    era5_avg[var_name] = var_avg

era5_avg.attrs = era5.attrs

era5_df = era5_avg.to_dataframe().reset_index()
#rename rime column to match the modis time column
era5_df = era5_df.rename(columns={'valid_time' : 'time'})
# era5_df['time']
# era5_avg.to_netcdf('ds_teh.nc')
# print(era5_avg.data_vars)

# era5_df.describe()
# era5_df.isnull().sum()

# terra_2022_df = aqua_2022.to_dataframe()
# terra_2022_df.isnull().sum()


In [3]:
#merging the MODIS datasets

terra_2022_df = terra_2022[['time', 'MOD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean']].to_dataframe().reset_index()
terra_2023_df = terra_2023[['time', 'MOD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean']].to_dataframe().reset_index()
aqua_2022_df = aqua_2022[['time', 'MYD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean']].to_dataframe().reset_index()
aqua_2023_df = aqua_2023[['time', 'MYD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean']].to_dataframe().reset_index()

terra_df = pd.concat([terra_2022_df, terra_2023_df], ignore_index=True)
aqua_df = pd.concat([aqua_2022_df, aqua_2023_df], ignore_index=True)

terra_df = terra_df.rename(columns={'MOD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean' : 'terra_aod'})
aqua_df = aqua_df.rename(columns={'MYD08_D3_6_1_Deep_Blue_Aerosol_Optical_Depth_550_Land_Mean' : 'aqua_aod'})

merged_modis_df = pd.merge(terra_df, aqua_df, on='time', how='outer')

merged_modis_df['aod'] = merged_modis_df.apply(
    lambda row: (row['terra_aod'] + row['aqua_aod']) / 2 if pd.notnull(row['terra_aod']) and pd.notnull(row['aqua_aod'])
    else row['terra_aod'] if pd.notnull(row['terra_aod'])
    else row['aqua_aod'] if pd.notnull(row['aqua_aod'])
    else np.nan,
    axis=1
)
#drop unneccessary columns
merged_modis_df = merged_modis_df.drop(columns=['terra_aod', 'aqua_aod'])

#adjust hours
merged_modis_df['time'] = merged_modis_df['time'] + pd.Timedelta(hours=12)
merged_modis_df

,time,aod
0,2022-01-01 12:00:00,0.1880
1,2022-01-02 12:00:00,NaN
2,2022-01-03 12:00:00,0.0940
3,2022-01-04 12:00:00,0.0535
4,2022-01-05 12:00:00,0.1835
...,...,...
725,2023-12-27 12:00:00,0.0675
726,2023-12-28 12:00:00,0.0620
727,2023-12-29 12:00:00,0.1135
728,2023-12-30 12:00:00,0.1325


In [8]:
final_df = pd.merge(era5_df, merged_modis_df, on='time', how='outer')
# final_df.isnull().describe()
# final_df.dropna().isnull().describe()
# final_df = final_df.dropna()
# final_df = final_df.drop
# final_df = final_df.drop(columns=['latitude', 'longitude', 'number', 'expver'])

In [14]:
# Convert back to xarray
# First, create a list of ERA5 variable names (excluding time/coordinates)
era5_vars = [var for var in era5.data_vars]

final_ds = xr.Dataset(
    {
        # First add ERA5 variables
        **{var: ('time', final_df[var]) for var in era5_vars},
        # Then add the combined AOD
        'aod': ('time', merged_modis_df['aod'])
    },
    coords={
        'time': merged_modis_df['time']
    },
    attrs={
        'description': 'Merged ERA5 and MODIS AOD dataset',
        'creation_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
        'time_zone': 'UTC',
        'region': '51E-52E, 35N-36N',
        'aod_description': 'Combined Terra and Aqua MODIS AOD, averaged when both available'
    }
)


In [15]:
# print(final_df)
# print(final_df.drop(columns='cbh').dropna())

In [16]:
print(final_ds)

<xarray.Dataset> Size: 53kB
Dimensions:  (time: 730)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2022-01-01T12:00:00 ... 2023-12-31T12:...
Data variables: (12/15)
    u10      (time) float32 3kB 0.7495 0.9964 2.014 ... -1.207 0.1727 1.235
    v10      (time) float32 3kB -0.3117 -0.3145 -1.727 ... 0.766 0.9249 0.7981
    t2m      (time) float32 3kB 280.4 278.9 279.9 279.3 ... 284.5 284.1 284.6
    sp       (time) float32 3kB 8.504e+04 8.454e+04 ... 8.565e+04 8.534e+04
    tp       (time) float32 3kB 5.722e-08 3.624e-07 3.368e-05 ... 0.0 0.0
    slhf     (time) float32 3kB -9.818e+04 -7.076e+04 ... -3.426e+04 -3.193e+04
    ...       ...
    tcc      (time) float32 3kB 0.9193 1.0 0.9788 0.0 ... 0.07519 0.003395 0.0
    blh      (time) float32 3kB 712.5 563.4 811.0 816.3 ... 846.1 619.6 573.6
    zust     (time) float32 3kB 0.105 0.1002 0.2056 ... 0.154 0.1087 0.1156
    tco3     (time) float32 3kB 0.007271 0.006591 0.0066 ... 0.006386 0.006543
    tcwv     (time) float32 3kB 7.13

In [17]:
final_ds.to_netcdf('final_merged_dataset.nc')

In [18]:
print(xr.open_dataset('final_merged_dataset.nc'))

<xarray.Dataset> Size: 53kB
Dimensions:  (time: 730)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2022-01-01T12:00:00 ... 2023-12-31T12:...
Data variables: (12/15)
    u10      (time) float32 3kB ...
    v10      (time) float32 3kB ...
    t2m      (time) float32 3kB ...
    sp       (time) float32 3kB ...
    tp       (time) float32 3kB ...
    slhf     (time) float32 3kB ...
    ...       ...
    tcc      (time) float32 3kB ...
    blh      (time) float32 3kB ...
    zust     (time) float32 3kB ...
    tco3     (time) float32 3kB ...
    tcwv     (time) float32 3kB ...
    aod      (time) float64 6kB ...
Attributes:
    description:      Merged ERA5 and MODIS AOD dataset
    creation_date:    2024-11-23
    time_zone:        UTC
    region:           51E-52E, 35N-36N
    aod_description:  Combined Terra and Aqua MODIS AOD, averaged when both a...
